# Synthetic data generation

Generate synthetic data by sampling parameter values from Pareto distributions (for behaviour parameters of political engagement and lead user popularity) and Gaussian distributions (for ideal points).

In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'notebook'
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
from typing import Union, Optional
from scipy.stats import norm
import jsonlines
import pathlib
import pickle
import math
import random
import time
from tabulate import tabulate
from sklearn.utils import check_random_state

In [2]:
seed_value = 1812
random.seed(seed_value)
np.random.seed(seed_value)
check_random_state(seed_value)

RandomState(MT19937) at 0x75D734C96C40

## Data Generation Functions

In [3]:
def generate_pareto_data(n_samples, n_dimensions, alpha=3):
    """
    Generate multi-dimensional Pareto distributed data.
    
    Parameters:
    - n_samples: Number of data points
    - n_dimensions: Number of dimensions
    - alpha: Shape parameter for Pareto distribution
    """
    return np.random.pareto(alpha, size=(n_samples, n_dimensions)).flatten().tolist()

def generate_normal_data(n_samples, n_dimensions, mu=0, sigma=1):
    """
    Generate multi-dimensional normally distributed data.
    
    Parameters:
    - n_samples: Number of data points
    - n_dimensions: Number of dimensions
    - mu: Mean of the normal distribution
    - sigma: Standard deviation of the normal distribution
    """
    rng = np.random.default_rng()
    if n_dimensions == 1:
        return rng.normal(mu, sigma, n_samples)
    else:
        return rng.multivariate_normal(mu, sigma, size=n_samples)

## Visualisation Functions

In [4]:
def create_distribution_plots(data, title, max_dims=3):
    """
    Create scatter plots for the first 2 or 3 dimensions of the data.
    """
    dims = min(data.shape[1], max_dims)
    
    if dims == 1:
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            y=data[:, 0],            
            mode='markers',
            marker=dict(size=5),
            name=title
        ))
        fig.update_layout(
            title=f'{title} Distribution (1D)',            
            yaxis_title='Dimension 1'
        )
    elif dims == 2:
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=data[:, 0],
            y=data[:, 1],
            mode='markers',
            marker=dict(size=5),
            name=title
        ))
        fig.update_layout(
            title=f'{title} Distribution (2D)',
            xaxis_title='Dimension 1',
            yaxis_title='Dimension 2'
        )
    else:
        fig = go.Figure(data=[go.Scatter3d(
            x=data[:, 0],
            y=data[:, 1],
            z=data[:, 2],
            mode='markers',
            marker=dict(size=3),
            name=title
        )])
        fig.update_layout(
            title=f'{title} Distribution (3D)',
            scene=dict(
                xaxis_title='Dimension 1',
                yaxis_title='Dimension 2',
                zaxis_title='Dimension 3'
            )
        )
    
    return fig

def create_histogram_matrix(data, title):
    """
    Create a matrix of histograms for each dimension.
    """
    n_dims = data.shape[1]
    fig = make_subplots(rows=n_dims, cols=1, subplot_titles=[f'Dimension {i+1}' for i in range(n_dims)])
    
    for i in range(n_dims):
        fig.add_trace(
            go.Histogram(x=data[:, i], name=f'Dim {i+1}', nbinsx=30),
            row=i+1, col=1
        )
    
    fig.update_layout(
        height=300 * n_dims,
        title_text=f'{title} - Histogram per Dimension',
        showlegend=False
    )
    
    return fig


def plot_array_heatmap(
    array: np.ndarray,
    title: str = "Array Heatmap",
    colorscale: Optional[Union[str, list]] = None,
    show_scale: bool = True,
    boundcolorscale: bool = False,
    x_labels: Optional[list] = None,
    y_labels: Optional[list] = None,
    width: int = 800,
    height: int = 600,
    annotation_format: str = ".2f", xtitle="Columns", ytitle="Rows", show_values=False
):
    """
    Creates a heatmap visualization of a 2D array using plotly.
    Automatically detects binary arrays and uses black/white colorscale.

    Parameters:
    -----------
    array : np.ndarray
        2D array to visualize
    title : str, optional
        Title of the heatmap
    colorscale : str or list, optional
        Custom colorscale. If None, uses 'black/white' for binary arrays
        and 'Viridis' for non-binary arrays
    show_scale : bool, optional
        Whether to show the colorbar
    x_labels : list, optional
        Custom labels for x-axis
    y_labels : list, optional
        Custom labels for y-axis
    width : int, optional
        Width of the figure in pixels
    height : int, optional
        Height of the figure in pixels
    annotation_format : str, optional
        Format string for cell annotations (e.g., ".2f" for 2 decimal places)
    """
    
    # Input validation
    if not isinstance(array, np.ndarray):
        array = np.array(array)
    
    if array.ndim != 2:
        raise ValueError("Input array must be 2-dimensional")

    # Check if array is binary
    is_binary = np.array_equal(array, array.astype(bool))
    
    # Set default colorscale based on array type
    if colorscale is None:
        colorscale = ['white', 'black'] if is_binary else 'Viridis'
    
    # Create axis labels if not provided
    if x_labels is None:
        x_labels = [str(i) for i in range(array.shape[1])]
    if y_labels is None:
        y_labels = [str(i) for i in range(array.shape[0])]
    
    # Format annotations based on binary/continuous values
    if is_binary:
        text = array.astype(int).astype(str)
    else:
        text = np.array([[f"{x:{annotation_format}}" for x in row] for row in array])
    
    # Create the heatmap
    if boundcolorscale:
        fig = go.Figure(data=go.Heatmap(
            z=array,
            x=x_labels,
            y=y_labels,
            colorscale=colorscale,
            showscale=show_scale,
            zmid=0.5,    # Center the colorscale at 0
            zmin=0,   # Set minimum value
            zmax=1, 
            colorbar=dict(            
                tickmode='array',
                tickvals=[-1, -0.5, 0, 0.5, 1],
                ticktext=['-1.0', '-0.5', '0.0', '0.5', '1.0']
                ),
            text=text if show_values else None,
            texttemplate="%{text}",
            textfont={"size": 10},
            hoverongaps=False,
            hovertemplate="x: %{x}<br>y: %{y}<br>value: %{z}<extra></extra>"
        ))
    else:
        fig = go.Figure(data=go.Heatmap(
            z=array,
            x=x_labels,
            y=y_labels,
            colorscale=colorscale,
            showscale=show_scale,            
            text=text if show_values else None,
            texttemplate="%{text}",
            textfont={"size": 10},
            hoverongaps=False,
            hovertemplate="x: %{x}<br>y: %{y}<br>value: %{z}<extra></extra>"
        ))
    
    # Update layout
    fig.update_layout(
        title=dict(
            text=title,
            x=0.5,
            xanchor='center'
        ),
        width=width,
        height=height,
        xaxis=dict(
            title=xtitle,
            side="bottom"
        ),
        yaxis=dict(
            title=ytitle,
            autorange="reversed"  # To match matrix orientation
        ),
        paper_bgcolor='white',
        plot_bgcolor='white'
    )
    
    return fig


def plot_side_by_side_subplots(fig1, fig2, fig3, title="Subplots"):
    """
    Arrange three Plotly figures side by side using subplots.
    
    Parameters:
    fig1, fig2, fig3: Plotly figure objects
    title: Main title for the combined plot
    """
    # Create subplot figure
    if fig2 is not None:
        fig = make_subplots(
            rows=1, 
            cols=3,
            subplot_titles=[fig1.layout.title.text, 
                           fig2.layout.title.text, 
                           fig3.layout.title.text]
        )
    else:
        fig = make_subplots(
            rows=1, 
            cols=2,
            subplot_titles=[fig1.layout.title.text,  
                           fig3.layout.title.text]
        )
        
    
    # Add traces from each figure
    for trace in fig1.data:
        fig.add_trace(trace, row=1, col=1)
    if fig2 is not None:
        for trace in fig2.data:
            fig.add_trace(trace, row=1, col=2)
        for trace in fig3.data:
            fig.add_trace(trace, row=1, col=3)
    else:
        for trace in fig3.data:
            fig.add_trace(trace, row=1, col=2)
        
    
    # Update layout
    fig.update_layout(
        title=dict(
            text=title,
            x=0.5,
            y=0.95,
            font=dict(size=24)
        ),
        showlegend=False,
        height=500,
        width=1200,
        template="plotly_white"
    )
    
    return fig


def create_scatter_plot(data_sets, labels=None, colors=None, sizes=None, symbols=None, title="Interactive Scatter Plot"):
    """
    Create an interactive scatter plot using Plotly for multiple sets of 2D data with customizable appearance.
    
    Parameters:
    data_sets (list): List of tuples, each containing (x_data, y_data) for each dataset
    labels (list): List of strings for legend labels
    colors (list): List of colors for each dataset
    sizes (list): List of marker sizes for each dataset
    symbols (list): List of marker symbols for each dataset
    title (str): Title of the plot
    """
    
    # Set default values if not provided
    n_sets = len(data_sets)
    if labels is None:
        labels = [f"Dataset {i+1}" for i in range(n_sets)]
    if colors is None:
        colors = ['blue', 'red', 'green']        
    if sizes is None:
        sizes = [10] * n_sets  # Plotly uses different size scale than matplotlib
    if symbols is None:
        symbols = ['circle', 'square', 'diamond']
    print(labels, colors, symbols, sizes)
    # Create figure
    fig = go.Figure()
    
    # Add each dataset as a separate trace
    for i, ((x_data, y_data), label, color, size, symbol) in enumerate(
            zip(data_sets, labels, colors, sizes, symbols)):
        fig.add_trace(
            go.Scatter(
                x=x_data,
                y=y_data,
                mode='markers',
                name=label,
                marker=dict(
                    size=size,
                    color=color,
                    symbol=symbol,
                    opacity=0.7,
                    line=dict(width=1, color='DarkSlateGrey')
                ),
                hovertemplate=
                f"{label}<br>" +
                "X: %{x:.2f}<br>" +
                "Y: %{y:.2f}<br>" +
                "<extra></extra>"  # This removes the secondary box in the hover tooltip
            )
        )
    
    # Update layout with more customization options
    fig.update_layout(
        title=dict(
            text=title,
            x=0.5,  # Center the title
            font=dict(size=24)
        ),
        xaxis=dict(
            title="X-axis",
            title_font=dict(size=14),
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGrey',
            zeroline=True,
            zerolinewidth=2,
            zerolinecolor='LightGrey'
        ),
        yaxis=dict(
            title="Y-axis",
            title_font=dict(size=14),
            showgrid=True,
            gridwidth=1,
            gridcolor='LightGrey',
            zeroline=True,
            zerolinewidth=2,
            zerolinecolor='LightGrey'
        ),
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99,
            bgcolor='rgba(255, 255, 255, 0.8)'
        ),
        hovermode='closest',
        plot_bgcolor='white'
    )
    
    
    return fig  # Return the figure object for potential further modifications

def fix_plot_layout_and_save(fig, savename, xaxis_title="", yaxis_title="", title="", showgrid=False, showlegend=False,
                             print_png=True, print_html=True, print_pdf=True):
        fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
        fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
        fig.update_layout(title=title, plot_bgcolor='rgb(255,255,255)',
                        yaxis=dict(
                            title=yaxis_title,
                            titlefont_size=20,
                            tickfont_size=20,
                            showgrid=showgrid,
                        ),
                        xaxis=dict(
                            title=xaxis_title,
                            titlefont_size=20,
                            tickfont_size=20,
                            showgrid=showgrid
                        ),
                        font=dict(
                            size=20
                        ),
                        showlegend=showlegend)
        if showlegend:
            fig.update_layout(legend=dict(
                yanchor="top",
                y=1.1,  # 0.01
                xanchor="right",  # "left", #  "right"
                x=1,    #0.01,  # 0.99
                bordercolor="Black",
                borderwidth=0.3,
                font=dict(
                    size=18,
        )))

        if print_html:
            pio.write_html(fig, savename, auto_open=False)
        if print_pdf:
            pio.write_image(fig, savename.replace(".html", ".pdf"), engine="kaleido")
        if print_png:
            pio.write_image(fig, savename.replace("html", "png"), width=1540, height=871, scale=1)

## Input parameters - model specifications

In [5]:
def generate_trial_data(m, J, K, mu_e, sigma_e, gamma, delta, alpha_js, beta_is, dtilde, xs, zs, phis, distance_func, utility_func, data_location):
    
    sigma_noise = sigma_e*np.eye(J)
    stochastic_component = generate_normal_data(n_samples=K, n_dimensions=J, mu=mu_e*np.ones((J,)), sigma=sigma_noise)
    utilities_matrix = np.zeros((K, J))    
    # assuming linear utility in this formulation
    # for i in range(K):
    #     for j in range(J):
    #         utilities_matrix[i,j] = (gamma*distance_func(xs[i, :], zs[j, :]) - delta*distance_func(xs[i, :], phis[j, :]) + alpha_js[j] + beta_is[i]) + stochastic_component[i, j]
    
    # assuming linear utility and euclidean distance in this formulation
    X_squared_norms = np.sum(xs.transpose()**2, axis=0).reshape(-1, 1)
    Z_squared_norms = np.sum(zs.transpose()**2, axis=0).reshape(1, -1)
    XZ_cross_terms = np.dot(xs, zs.transpose())    
    Phi_squared_norms = np.sum(phis.transpose()**2, axis=0).reshape(1, -1)
    XPhi_cross_terms = np.dot(xs, phis.transpose())
    XZ_distances = np.sqrt(X_squared_norms + Z_squared_norms - 2 * XZ_cross_terms)
    XPhi_distances = np.sqrt(X_squared_norms + Phi_squared_norms - 2 * XPhi_cross_terms)
    utilities_matrix = gamma*XZ_distances - delta*XPhi_distances + np.tile(alpha_js, (K, 1)) + np.repeat(np.array(beta_is).reshape(-1, 1), J, axis=1) + stochastic_component
    # assert(np.allclose(utilities, utilities_matrix))

    # utilities_matrix = generate_normal_data(n_samples=K, n_dimensions=J, mu=0.6*np.ones((J,)), sigma=0.1*np.eye(J))
    
    utilities_mat_probab = norm.cdf(utilities_matrix, loc=mu_e, scale=sigma_e)
    follow_matrix = utilities_mat_probab > 0.5
       
    print(tabulate(utilities_matrix))
    
    # save data
    with open("{}/Y.pickle".format(data_location), "wb") as f:
        pickle.dump(follow_matrix, f, protocol=4)

    # full, with status quo
    if delta > 0:
        parameter_space_dim = (K+2*J)*dtilde + J + K + 4
    else:
        # no status quo
        parameter_space_dim = (K+J)*dtilde + J + K + 3
    # for distributing per N rows
    print("Parameter space dimensionality: {}".format(parameter_space_dim))
    N = math.ceil(parameter_space_dim/J)
    print("Subset row number: {}".format(N))
    print("Observed data points per data split: {}".format(N*J))
    # subset rows (users)   
    subset_dataset_size = N
    for i in range(0, K, N):
        from_row = i 
        to_row = np.min([i+N, K])
        # print(from_row, to_row)
        if i+2*N > K:
            to_row = K
        pathlib.Path("{}/dataset_{}_{}".format(data_location, from_row, to_row)).mkdir(parents=True, exist_ok=True)
        with open("{}/dataset_{}_{}/dataset_{}_{}.pickle".format(data_location, from_row, to_row, from_row, to_row), "wb") as f:
            pickle.dump(follow_matrix[from_row:to_row, :], f, protocol=4)             
        fig = plot_array_heatmap(
            utilities_matrix[from_row:to_row, :],
            title="Computed utilities",
            colorscale="sunsetdark", boundcolorscale=True,
            xtitle="Leaders", ytitle="Followers", show_values=False, show_scale=True
            )  
        probabfig = plot_array_heatmap(
            utilities_mat_probab[from_row:to_row, :],
            title="Pij CDF matrix",
            colorscale="Viridis",
            xtitle="Leaders", ytitle="Followers", show_values=False, show_scale=False
            )  
        followfig = plot_array_heatmap(
                follow_matrix[from_row:to_row, :].astype(np.int8),
                title="Following",
                xtitle="Leaders", ytitle="Followers", show_values=False, show_scale=False
            )    
        allplots = plot_side_by_side_subplots(fig, None, followfig, title="Synthetic data")    
        fix_plot_layout_and_save(allplots, "{}/dataset_{}_{}/utilities_following_relationships.html".format(data_location, from_row, to_row), 
                                xaxis_title="", yaxis_title="", title="", showgrid=False, showlegend=False,
                                print_png=True, print_html=True, print_pdf=False)
        # print(np.sum(follow_matrix[from_row:to_row, :], axis=1))
        # print(np.sum(follow_matrix[from_row:to_row, :], axis=0))
        
        if i+2*N > K:
            break

    # plots
    fig = plot_array_heatmap(
        utilities_matrix,
        title="Computed utilities",
        colorscale="Viridis", boundcolorscale=True,
        xtitle="Leaders", ytitle="Followers", show_values=False, show_scale=True
    )    
    followfig = plot_array_heatmap(
            follow_matrix.astype(np.int8),
            title="Following",
            xtitle="Leaders", ytitle="Followers", show_values=False, show_scale=False
        )    
    allplots = plot_side_by_side_subplots(fig, None, followfig, title="Synthetic data")    
    fix_plot_layout_and_save(allplots, "{}/utilities_following_relationships.html".format(data_location), xaxis_title="", yaxis_title="", title="", showgrid=False, showlegend=False,
                                 print_png=True, print_html=True, print_pdf=False)

    if delta > 0:
        fig = create_scatter_plot(
                data_sets=[(xs[:, 0], xs[:, 1]), (zs[:, 0], zs[:, 1]), (phis[:, 0], phis[:, 1])],
                labels=["Followers", "Leaders", "Status quo"],
                colors=["blue", "orange", "green"],
                sizes=[8, 12, 10],
                symbols=["circle", "diamond", "star"],
                title=""
            )
    else:
        fig = create_scatter_plot(
                data_sets=[(xs[:, 0], xs[:, 1]), (zs[:, 0], zs[:, 1])],
                labels=["Followers", "Leaders"],
                colors=["blue", "orange"],
                sizes=[8, 12],
                symbols=["circle", "diamond"],
                title=""
            )
    fig.layout.height = 700    
    fix_plot_layout_and_save(fig, "{}/network_users_vis.html".format(data_location), xaxis_title="", yaxis_title="", title="", showgrid=False, showlegend=False,
                                 print_png=True, print_html=True, print_pdf=False)

In [6]:
# number of leaders
J = 10
# number of followers
K = 30

# trials
M = 1

mu_e = 0
sigma_e = 0.0001

# utility model parameters
gamma = -1.0
delta = 0.0
# leaders popularity
alpha_mean = 0.0
alpha_var = 0.25
alpha_js = [generate_normal_data(n_samples=1, n_dimensions=1, mu=alpha_mean, sigma=alpha_var)[0] for i in range(J)] #generate_pareto_data(n_samples=J-1, n_dimensions=1, alpha=1)
# followers' political interest
beta_mean = 1
beta_var = 0.25
beta_is = [generate_normal_data(n_samples=1, n_dimensions=1, mu=beta_mean, sigma=beta_var)[0] for i in range(K)] #generate_pareto_data(n_samples=K, n_dimensions=1, alpha=3)

# dimensionality of ideal points space
dtilde = 2

# good visual, incompatible with social science perspective
# leaders' ideal points
# mu = lambda x: np.asarray([generate_normal_data(n_samples=1, n_dimensions=1, mu=np.asarray([1.5+x]), sigma=5), 
#                             generate_normal_data(n_samples=1, n_dimensions=1, mu=np.asarray([-1.5-x]), sigma=3)]).reshape((dtilde,))  
# zs = np.vstack([generate_normal_data(n_samples=1, n_dimensions=dtilde, mu=mu(j), sigma=np.eye(dtilde)) for j in range(J)])
# # followers' ideal points - place their mean randomly on the leaders
# zidx = np.arange(0, J)
# mu_xs = np.vstack(zs[np.random.choice(zidx, size=K, replace=True),:])
# xs = np.vstack([generate_normal_data(n_samples=1, n_dimensions=dtilde, mu=mu_xs[i,:], sigma=np.eye(dtilde)) for i in range(K)])
# # status quo ideal points
# # phis = generate_normal_data(n_samples=J, n_dimensions=dtilde, mu=np.zeros((dtilde,)), sigma=np.eye(dtilde))
# mu_phis = np.asarray([generate_normal_data(n_samples=1, n_dimensions=1, mu=np.asarray([-2.5]), sigma=1), 
#                         generate_normal_data(n_samples=1, n_dimensions=1, mu=np.asarray([2.5]), sigma=1)]).reshape((dtilde,))
# phis = generate_normal_data(n_samples=J, n_dimensions=dtilde, mu=mu_phis, sigma=np.eye(dtilde))

# followers' ideal points
xs_mean = np.zeros((dtilde,))
xs_sigma = 0.1*np.eye(dtilde)
xs = generate_normal_data(n_samples=K, n_dimensions=dtilde, mu=xs_mean, sigma=xs_sigma)
# leaders' ideal points - unimodal distribution
zs_mean = np.zeros((dtilde,))
zs_sigma = 0.1*np.eye(dtilde)
zs = generate_normal_data(n_samples=J, n_dimensions=dtilde, mu=zs_mean, sigma=zs_sigma)
# status quo ideal points
phis_mean = np.zeros((dtilde,))
phis_sigma = 0.5*np.eye(dtilde)
phis = generate_normal_data(n_samples=J, n_dimensions=dtilde, mu=phis_mean, sigma=phis_sigma)

# utility and distance functions
distance_func = lambda x,y : np.linalg.norm(x-y,ord=2)
utility_func = lambda x : x

In [7]:
# followers' political interest
# for alphastep in np.arange(0.0,0.1,0.5):
#     alpha_js = [generate_normal_data(n_samples=1, n_dimensions=1, mu=alphastep, sigma=0.1)[0] for i in range(J)] #generate_pareto_data(n_samples=J-1, n_dimensions=1, alpha=1)    
#     for betastep in np.arange(0,1.0,0.1):
#         print(alphastep, betastep)
#         beta_is = [generate_normal_data(n_samples=1, n_dimensions=1, mu=betastep, sigma=0.1)[0] for i in range(K)] #generate_pareto_data(n_samples=K, n_dimensions=1, alpha=3)
        
# generate data for M trials
for m in range(M):
    data_location = "/home/ioannischalkiadakis/ideal/idealpestimation/data_K{}_J{}_sigmae{}_nopareto/{}/".format(K, J, str(sigma_e).replace(".", ""), m)
    # data_location = "/home/ioannis/Dropbox (Heriot-Watt University Team)/ideal/idealpestimation/data_K{}_J{}_sigmae{}_nopareto/{}/".format(K, J, str(sigma_e).replace(".", ""), m)
    pathlib.Path(data_location).mkdir(parents=True, exist_ok=True) 
    generate_trial_data(m, J, K, mu_e, sigma_e, gamma, delta, alpha_js, beta_is, dtilde, xs, zs, phis, distance_func, utility_func, data_location)
    # time.sleep(2)

----------  ----------  ----------  ---------  ----------  ----------  -----------  ----------  ---------  ----------
 0.0495233   0.99088     0.575798    0.571293   0.213046    0.179854    0.274004     0.243103    0.723635  -0.457181
-0.08511     0.709366    0.413518    0.40077   -0.0296646   0.0228415   0.147046     0.0788118   0.577629  -0.630554
 0.621398    1.08474     1.16352     1.04636    0.410772    0.639337    0.892561     0.596563    1.08326    0.115298
 0.621407    0.85325     0.976669    0.936503   0.337717    0.632063    0.858297     0.515336    0.97296    0.119053
 0.645596    0.736271    0.698554    0.965579   0.647604    0.83418     0.682758     0.819708    1.08577    0.116509
 0.0852096   0.651286    0.332453    0.498295   0.815829    0.426446    0.169047     0.487398    0.708925  -0.398231
 0.255271    0.917705    0.584524    0.723715   0.612699    0.455792    0.400727     0.547812    0.915742  -0.312427
 0.669505    1.01209     0.898977    1.10302    0.803246    0.8

In [ ]:
# m = 2
# data_location = "/home/ioannischalkiadakis/ideal/idealpestimation/data_K{}_J{}_sigmae{}/{}/".format(K, J, str(sigma_e).replace(".", ""), m)
# pathlib.Path(data_location).mkdir(parents=True, exist_ok=True) 
# generate_trial_data(m, J, K, mu_e, sigma_e, gamma, delta, alpha_js, beta_is, dtilde, xs, zs, phis, distance_func, utility_func, data_location)

### Save parameters

In [9]:
parameters = dict()
parameters["J"] = J
parameters["K"] = K
parameters["mu_e"] = mu_e
parameters["sigma_e"] = sigma_e
parameters["gamma"] = gamma
parameters["delta"] = delta
parameters["alpha"] = alpha_js
parameters["alpha_mean"] = alpha_mean
parameters["alpha_cov"] = alpha_var
parameters["beta"] = beta_is
parameters["beta_mean"] = beta_mean
parameters["beta_cov"] = beta_var
parameters["d"] = dtilde
if dtilde > 1:
    parameters["xs_mean"] = xs_mean.tolist()
    parameters["xs_sigma"] = xs_sigma.reshape((dtilde*dtilde,), order="F").flatten().tolist()
    parameters["zs_mean"] = zs_mean.tolist()
    parameters["zs_sigma"] = zs_sigma.reshape((dtilde*dtilde,), order="F").flatten().tolist()
    parameters["phis_mean"] = phis_mean.tolist()
    parameters["phis_sigma"] = phis_sigma.reshape((dtilde*dtilde,), order="F").flatten().tolist()
else:
    parameters["xs_mean"] = xs_mean
    parameters["xs_sigma"] = xs_sigma
    parameters["zs_mean"] = zs_mean
    parameters["zs_sigma"] = zs_sigma
    parameters["phis_mean"] = phis_mean
    parameters["phis_sigma"] = phis_sigma

parameters["Z"] = zs.reshape((dtilde*J,), order="F").flatten().tolist()
parameters["X"] = xs.reshape((dtilde*K,), order="F").flatten().tolist()
parameters["Phi"] = phis.reshape((dtilde*J,), order="F").flatten().tolist()
# print(parameters)
DATA_dir = "/home/ioannischalkiadakis/ideal/idealpestimation/data_K{}_J{}_sigmae{}_nopareto/".format(K, J, str(sigma_e).replace(".", ""))
# DATA_dir = "/home/ioannis/Dropbox (Heriot-Watt University Team)/ideal/idealpestimation/data_K{}_J{}_sigmae{}_nopareto/".format(K, J, str(sigma_e).replace(".", ""))
pathlib.Path(DATA_dir).mkdir(parents=True, exist_ok=True)     
with jsonlines.open("{}/synthetic_gen_parameters.jsonl".format(DATA_dir), "a") as f:
    f.write(parameters)

## Compute utilities

In [ ]:
utilities_matrix = np.zeros((K, J))
# sigma_noise = 0.5*np.eye(J)
# stochastic_component = generate_normal_data(n_samples=K, n_dimensions=J, mu=np.zeros((J,)), sigma=sigma_noise)

# assuming linear utility in this formulation
for i in range(K):
    for j in range(J):
        utilities_matrix[i,j] = (gamma*distance_func(xs[i, :], zs[j, :]) - delta*distance_func(xs[i, :], phis[j, :]) + alpha_js[j] + beta_is[i]) + generate_normal_data(n_samples=1, n_dimensions=1, mu=0, sigma=0.5)[0]
        # print(xs[i, :], zs[j, :], phis[j, :], alpha_js[j] + beta_is[i]) + generate_normal_data(n_samples=1, n_dimensions=1, mu=0, sigma=0.5)
        # print(distance_func(xs[i, :], zs[j, :]))
       
# utilities_matrix += stochastic_component
utilities_mat_probab = norm.cdf(utilities_matrix, loc=mu_e, scale=sigma_e)

## Plot

In [ ]:
fig = plot_array_heatmap(
        utilities_matrix,
        title="Computed utilities",
        colorscale="Viridis",
        xtitle="Leaders", ytitle="Followers", show_values=False, show_scale=False
    )
# fig.show()
# noisefig = plot_array_heatmap(
#         stochastic_component,
#         title="Error component",
#         colorscale="Viridis",
#         show_values=False, show_scale=False
#     )
# noisefig.show()
follow_matrix = utilities_mat_probab > 0.5
followfig = plot_array_heatmap(
        follow_matrix.astype(np.int8),
        title="Following",
        xtitle="Leaders", ytitle="Followers", show_values=False, show_scale=False
    )


# followfig.show()
allplots = plot_side_by_side_subplots(fig, None, followfig, title="Synthetic data")
allplots.show()
fix_plot_layout_and_save(allplots, "{}/utilities_following_relationships.html".format(DATA_dir), xaxis_title="", yaxis_title="", title="", showgrid=False, showlegend=False,
                             print_png=True, print_html=True, print_pdf=True)

### Save observed data

In [ ]:
with open("{}/Y.pickle".format(DATA_dir), "wb") as f:
    pickle.dump(follow_matrix, f, protocol=4)

# full, with status quo
# parameter_space_dim = (K+2*J)*dtilde + J + K + 4
# no status quo
parameter_space_dim = (K+J)*dtilde + J + K + 3
# for distributing per N rows
print("Parameter space dimensionality: {}".format(parameter_space_dim))
N = math.ceil(parameter_space_dim/J)
print(N)
print("Observed data points per data split: {}".format(N*J))
# subset rows (users)
subset_dataset_size = N
for i in range(0, K, N):
    from_row = i 
    to_row = np.min([i+N, K])
    print(from_row, to_row)
    if i+2*N > K:
        to_row = K
    with open("{}/dataset_{}_{}.pickle".format(DATA_dir, from_row, to_row), "wb") as f:
        pickle.dump(follow_matrix[from_row:to_row, :], f, protocol=4)    
    if i+2*N > K:
        break

In [ ]:
if delta > 0:
    fig = create_scatter_plot(
            data_sets=[(xs[:, 0], xs[:, 1]), (zs[:, 0], zs[:, 1]), (phis[:, 0], phis[:, 1])],
            labels=["Followers", "Leaders", "Status quo"],
            colors=["blue", "orange", "green"],
            sizes=[8, 12, 10],
            symbols=["circle", "diamond", "star"],
            title=""
        )
else:
    fig = create_scatter_plot(
            data_sets=[(xs[:, 0], xs[:, 1]), (zs[:, 0], zs[:, 1])],
            labels=["Followers", "Leaders"],
            colors=["blue", "orange"],
            sizes=[8, 12],
            symbols=["circle", "diamond"],
            title=""
        )
fig.layout.height = 700
fig.show()
fix_plot_layout_and_save(fig, "{}/network_users_vis.html".format(DATA_dir), xaxis_title="", yaxis_title="", title="", showgrid=False, showlegend=False,
                             print_png=True, print_html=True, print_pdf=True)